##Import Data

In [10]:
import pandas as pd

df = pd.read_csv('/content/labeltiket(Star).csv')
df.drop(columns=df.columns[[0, 1]], inplace=True)
df

,userName,content,score,at,label
0,Seo Hyeonjo,"Fitur nomor ponsel tidak bisa diubah, gimana k...",1,2022-09-11 00:23:11,-1
1,Dandy Iswan,Aplikasinya kurang nyaman digunakan. Pesanan y...,1,2022-09-01 06:30:29,-1
2,Hani Lukmanto,1). Harga menginap di hotel yg saya pilih lebi...,1,2022-09-16 10:40:20,-1
3,Rinipuha Handayani,"Sangat kecewa, saya mau pesen tiket timnas vs ...",1,2022-09-25 07:53:23,-1
4,RICARD CASTELO,"Sangat buruk, mengajukan payleter berbelit-bel...",1,2022-09-19 15:06:12,-1
...,...,...,...,...,...
935,Almair Dzaky Khan,"Paket buat travelingnya juga donk, biar semaki...",5,2022-04-02 21:05:00,1
936,wony eko apriyanto,Beberapa pesan disini sangat puas..mudah cepat...,5,2021-12-29 15:08:05,1
937,Toriyansah Toriyansah,Terlmakasi atas tiket memang nyata,5,2022-07-16 20:36:18,1
938,Joko sujatmiko ew,Sangat bermanfaat,4,2022-07-27 07:30:49,1


##Case Folding

In [11]:
!pip install sastrawi

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
df.drop(columns=df.columns[[0,2,3]], inplace=True)

In [13]:
def casefolding(text):
    return text.lower()

df['case_folding'] = df['content'].apply(lambda x: casefolding(x))

##Remove Emoticon, Punctuation, Number, Strip White Space

In [14]:
# Implementasi Regular Expression untuk menghapus simbol dan emoji
def unusedchar(text):
  text = re.sub(r'@[A-za-z0-9]+', '', text) #removing @mentions
  text = re.sub('[0-9]+', '', text) #removing number
  text = re.sub(r'#', '', text) #removing '#' symbol
  text = re.sub(r'RT[\s]+', '', text) #removing RT
  text = re.sub(r'#', '', text) #removing '#' symbol
  text = re.sub(r'[\n]+', '', text) #remove new line
  text = re.sub(r"^\s+|\s+$", "", text) #remove leading and trailing spaces in a word using OR sign to delete both
  text = re.sub(r" +", " ", text) #remove multiple space betwen words
  text = re.sub(r'http\S+', '', text) #yang ini yang buat remove https
  text = re.sub(r"\b[a-zA-Z]\b", "", text) #removing single char
  text = re.sub('\s+', ' ', text) #removing multiple whitespace
  text = re.sub(r'\W', " ", text)
  text = text.replace('\\t', ' ').replace('\\n', ' ').replace('\\u', ' ').replace('\\', ' ')#remove tab, new line, and back slice
  text = re.sub(r'[^\w\s]', ' ', text) #remove puntuation $ emoji (remove all besides \w > word dan \s > space)
  text = re.sub(r"²|¹|³|⁰|ⁱ|⁴|⁵|⁶|⁷|⁸|⁹|⁺|⁻|⁽|⁾|ⁿ]+", "", text) #ilangin pangkat
  text = re.sub(r'_', '',text) #ilangin garisbawah
  text = text.translate(str.maketrans("", "", string.punctuation)) #ilangin punctuation(,:';)
  return text

  # remove unusedchar
df['remove_unusedchar'] = df['case_folding'].apply(lambda x: unusedchar(x))

In [15]:
# Normalisasi kata dengan KBBI
dict = {row[0] : row[1] for _, row in pd.read_csv("kbba.txt", delimiter = "\t").iterrows()}

def kbbi(text): 
    token = text.split()
    final_string = ' '.join(str(dict.get(word, word)) for word in token)
    return final_string

df['KBBI'] = df['remove_unusedchar'].apply(lambda x: kbbi(x))
df

,content,label,case_folding,remove_unusedchar,KBBI
0,"Fitur nomor ponsel tidak bisa diubah, gimana k...",-1,"fitur nomor ponsel tidak bisa diubah, gimana k...",fitur nomor ponsel tidak bisa diubah gimana k...,fitur nomor telepon seluler tidak bisa diubah ...
1,Aplikasinya kurang nyaman digunakan. Pesanan y...,-1,aplikasinya kurang nyaman digunakan. pesanan y...,aplikasinya kurang nyaman digunakan pesanan y...,aplikasinya kurang nyaman digunakan pesanan ya...
2,1). Harga menginap di hotel yg saya pilih lebi...,-1,1). harga menginap di hotel yg saya pilih lebi...,harga menginap di hotel yg saya pilih lebih...,harga menginap di hotel yang saya pilih lebih ...
3,"Sangat kecewa, saya mau pesen tiket timnas vs ...",-1,"sangat kecewa, saya mau pesen tiket timnas vs ...",sangat kecewa saya mau pesen tiket timnas vs ...,sangat kecewa saya mau pesan tiket timnas vs c...
4,"Sangat buruk, mengajukan payleter berbelit-bel...",-1,"sangat buruk, mengajukan payleter berbelit-bel...",sangat buruk mengajukan payleter berbelit bel...,sangat buruk mengajukan fitur bayar nanti berb...
...,...,...,...,...,...
935,"Paket buat travelingnya juga donk, biar semaki...",1,"paket buat travelingnya juga donk, biar semaki...",paket buat travelingnya juga donk biar semaki...,paket buat berpergian juga dong biar semakin l...
936,Beberapa pesan disini sangat puas..mudah cepat...,1,beberapa pesan disini sangat puas..mudah cepat...,beberapa pesan disini sangat puas mudah cepat...,beberapa pesan disini sangat puas mudah cepat ...
937,Terlmakasi atas tiket memang nyata,1,terlmakasi atas tiket memang nyata,terlmakasi atas tiket memang nyata,terimakasih atas tiket memang nyata
938,Sangat bermanfaat,1,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat


In [16]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [17]:
# Menghapus baris yang mengandung noise
def noise(text): 
    text = text.split()
    listStopword =['dong','apaya','nya','g','y','vs','ada','adalah','adanya',
                   'adapun','agak', 'agaknya', 'agar', 'akan','aku','gua','lu',
                   'kamu','ayo','lah','yang', 'untuk', 'pada', 'ke', 'para', 
                   'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti',
                   'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini',
                   'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 
                   'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 
                   'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 
                   'adalah', 'itu', 'dalam', 'bahwa', 'atau', 'hanya', 
                   'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 
                   'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 
                   'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 
                   'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 
                   'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 
                   'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 
                   'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 
                   'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 
                   'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 
                   'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 
                   'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 
                   'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 
                   'saja', 'toh', 'ya', 'walau', 'tolong', 'tentu', 'amat', 
                   'apalagi', 'bagaimanapun','timnas','curacao','cs','oneklik','ovo',
                   'gopay','eh','di','eh','download','bbrp']
    text = [w for w in text if not w in listStopword]
    text = " ".join(text)
    return text

df['Stopword'] = df['KBBI'].apply(lambda x: noise(x))
df


,content,label,case_folding,remove_unusedchar,KBBI,Stopword
0,"Fitur nomor ponsel tidak bisa diubah, gimana k...",-1,"fitur nomor ponsel tidak bisa diubah, gimana k...",fitur nomor ponsel tidak bisa diubah gimana k...,fitur nomor telepon seluler tidak bisa diubah ...,fitur nomor telepon seluler tidak bisa diubah ...
1,Aplikasinya kurang nyaman digunakan. Pesanan y...,-1,aplikasinya kurang nyaman digunakan. pesanan y...,aplikasinya kurang nyaman digunakan pesanan y...,aplikasinya kurang nyaman digunakan pesanan ya...,aplikasinya kurang nyaman digunakan pesanan ma...
2,1). Harga menginap di hotel yg saya pilih lebi...,-1,1). harga menginap di hotel yg saya pilih lebi...,harga menginap di hotel yg saya pilih lebih...,harga menginap di hotel yang saya pilih lebih ...,harga menginap hotel pilih lebih mahal waktu c...
3,"Sangat kecewa, saya mau pesen tiket timnas vs ...",-1,"sangat kecewa, saya mau pesen tiket timnas vs ...",sangat kecewa saya mau pesen tiket timnas vs ...,sangat kecewa saya mau pesan tiket timnas vs c...,sangat kecewa mau pesan tiket tidak bisa bisa ...
4,"Sangat buruk, mengajukan payleter berbelit-bel...",-1,"sangat buruk, mengajukan payleter berbelit-bel...",sangat buruk mengajukan payleter berbelit bel...,sangat buruk mengajukan fitur bayar nanti berb...,sangat buruk mengajukan fitur bayar berbelit b...
...,...,...,...,...,...,...
935,"Paket buat travelingnya juga donk, biar semaki...",1,"paket buat travelingnya juga donk, biar semaki...",paket buat travelingnya juga donk biar semaki...,paket buat berpergian juga dong biar semakin l...,paket buat berpergian biar semakin lengkap
936,Beberapa pesan disini sangat puas..mudah cepat...,1,beberapa pesan disini sangat puas..mudah cepat...,beberapa pesan disini sangat puas mudah cepat...,beberapa pesan disini sangat puas mudah cepat ...,beberapa pesan disini sangat puas mudah cepat ...
937,Terlmakasi atas tiket memang nyata,1,terlmakasi atas tiket memang nyata,terlmakasi atas tiket memang nyata,terimakasih atas tiket memang nyata,terimakasih atas tiket memang nyata
938,Sangat bermanfaat,1,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat


In [18]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def Stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = text.split()
  hasil = [stemmer.stem(token) for token in text]
  hasil = " ".join(hasil)
  return hasil

df['Stemming'] = df['Stopword'].apply(Stemming)
df

,content,label,case_folding,remove_unusedchar,KBBI,Stopword,Stemming
0,"Fitur nomor ponsel tidak bisa diubah, gimana k...",-1,"fitur nomor ponsel tidak bisa diubah, gimana k...",fitur nomor ponsel tidak bisa diubah gimana k...,fitur nomor telepon seluler tidak bisa diubah ...,fitur nomor telepon seluler tidak bisa diubah ...,fitur nomor telepon seluler tidak bisa ubah ba...
1,Aplikasinya kurang nyaman digunakan. Pesanan y...,-1,aplikasinya kurang nyaman digunakan. pesanan y...,aplikasinya kurang nyaman digunakan pesanan y...,aplikasinya kurang nyaman digunakan pesanan ya...,aplikasinya kurang nyaman digunakan pesanan ma...,aplikasi kurang nyaman guna pesan masuk tahap ...
2,1). Harga menginap di hotel yg saya pilih lebi...,-1,1). harga menginap di hotel yg saya pilih lebi...,harga menginap di hotel yg saya pilih lebih...,harga menginap di hotel yang saya pilih lebih ...,harga menginap hotel pilih lebih mahal waktu c...,harga inap hotel pilih lebih mahal waktu coba ...
3,"Sangat kecewa, saya mau pesen tiket timnas vs ...",-1,"sangat kecewa, saya mau pesen tiket timnas vs ...",sangat kecewa saya mau pesen tiket timnas vs ...,sangat kecewa saya mau pesan tiket timnas vs c...,sangat kecewa mau pesan tiket tidak bisa bisa ...,sangat kecewa mau pesan tiket tidak bisa bisa ...
4,"Sangat buruk, mengajukan payleter berbelit-bel...",-1,"sangat buruk, mengajukan payleter berbelit-bel...",sangat buruk mengajukan payleter berbelit bel...,sangat buruk mengajukan fitur bayar nanti berb...,sangat buruk mengajukan fitur bayar berbelit b...,sangat buruk aju fitur bayar belit belit minta...
...,...,...,...,...,...,...,...
935,"Paket buat travelingnya juga donk, biar semaki...",1,"paket buat travelingnya juga donk, biar semaki...",paket buat travelingnya juga donk biar semaki...,paket buat berpergian juga dong biar semakin l...,paket buat berpergian biar semakin lengkap,paket buat pergi biar makin lengkap
936,Beberapa pesan disini sangat puas..mudah cepat...,1,beberapa pesan disini sangat puas..mudah cepat...,beberapa pesan disini sangat puas mudah cepat...,beberapa pesan disini sangat puas mudah cepat ...,beberapa pesan disini sangat puas mudah cepat ...,beberapa pesan sini sangat puas mudah cepat ba...
937,Terlmakasi atas tiket memang nyata,1,terlmakasi atas tiket memang nyata,terlmakasi atas tiket memang nyata,terimakasih atas tiket memang nyata,terimakasih atas tiket memang nyata,terimakasih atas tiket memang nyata
938,Sangat bermanfaat,1,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat,sangat manfaat


In [19]:
label = []
for index, row in df.iterrows():
    if row["label"] == -1:
        label.append('Negatif')
    else:
        label.append('Positif')

df["label"] = label
df

,content,label,case_folding,remove_unusedchar,KBBI,Stopword,Stemming
0,"Fitur nomor ponsel tidak bisa diubah, gimana k...",Negatif,"fitur nomor ponsel tidak bisa diubah, gimana k...",fitur nomor ponsel tidak bisa diubah gimana k...,fitur nomor telepon seluler tidak bisa diubah ...,fitur nomor telepon seluler tidak bisa diubah ...,fitur nomor telepon seluler tidak bisa ubah ba...
1,Aplikasinya kurang nyaman digunakan. Pesanan y...,Negatif,aplikasinya kurang nyaman digunakan. pesanan y...,aplikasinya kurang nyaman digunakan pesanan y...,aplikasinya kurang nyaman digunakan pesanan ya...,aplikasinya kurang nyaman digunakan pesanan ma...,aplikasi kurang nyaman guna pesan masuk tahap ...
2,1). Harga menginap di hotel yg saya pilih lebi...,Negatif,1). harga menginap di hotel yg saya pilih lebi...,harga menginap di hotel yg saya pilih lebih...,harga menginap di hotel yang saya pilih lebih ...,harga menginap hotel pilih lebih mahal waktu c...,harga inap hotel pilih lebih mahal waktu coba ...
3,"Sangat kecewa, saya mau pesen tiket timnas vs ...",Negatif,"sangat kecewa, saya mau pesen tiket timnas vs ...",sangat kecewa saya mau pesen tiket timnas vs ...,sangat kecewa saya mau pesan tiket timnas vs c...,sangat kecewa mau pesan tiket tidak bisa bisa ...,sangat kecewa mau pesan tiket tidak bisa bisa ...
4,"Sangat buruk, mengajukan payleter berbelit-bel...",Negatif,"sangat buruk, mengajukan payleter berbelit-bel...",sangat buruk mengajukan payleter berbelit bel...,sangat buruk mengajukan fitur bayar nanti berb...,sangat buruk mengajukan fitur bayar berbelit b...,sangat buruk aju fitur bayar belit belit minta...
...,...,...,...,...,...,...,...
935,"Paket buat travelingnya juga donk, biar semaki...",Positif,"paket buat travelingnya juga donk, biar semaki...",paket buat travelingnya juga donk biar semaki...,paket buat berpergian juga dong biar semakin l...,paket buat berpergian biar semakin lengkap,paket buat pergi biar makin lengkap
936,Beberapa pesan disini sangat puas..mudah cepat...,Positif,beberapa pesan disini sangat puas..mudah cepat...,beberapa pesan disini sangat puas mudah cepat...,beberapa pesan disini sangat puas mudah cepat ...,beberapa pesan disini sangat puas mudah cepat ...,beberapa pesan sini sangat puas mudah cepat ba...
937,Terlmakasi atas tiket memang nyata,Positif,terlmakasi atas tiket memang nyata,terlmakasi atas tiket memang nyata,terimakasih atas tiket memang nyata,terimakasih atas tiket memang nyata,terimakasih atas tiket memang nyata
938,Sangat bermanfaat,Positif,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat,sangat bermanfaat,sangat manfaat


In [20]:
df['clean'] = df['Stemming']
df.drop(columns=df.columns[[0,2,3,4,5,6]], inplace=True)

In [21]:
df.to_csv('clean_comment.csv')
df = pd.read_csv('/content/clean_comment.csv', sep=',')
df.drop(columns=df.columns[[0]], inplace=True)
df

,label,clean
0,Negatif,fitur nomor telepon seluler tidak bisa ubah ba...
1,Negatif,aplikasi kurang nyaman guna pesan masuk tahap ...
2,Negatif,harga inap hotel pilih lebih mahal waktu coba ...
3,Negatif,sangat kecewa mau pesan tiket tidak bisa bisa ...
4,Negatif,sangat buruk aju fitur bayar belit belit minta...
...,...,...
935,Positif,paket buat pergi biar makin lengkap
936,Positif,beberapa pesan sini sangat puas mudah cepat ba...
937,Positif,terimakasih atas tiket memang nyata
938,Positif,sangat manfaat


In [22]:
data = df.clean.values.tolist()
label = df.label.values.tolist()
print(data)
print(label)

['fitur nomor telepon seluler tidak bisa ubah bagaimana kalau ganti nomor telepon seluler aplikasi tidak bisa ganti metode bayar baik bisa simpan metode bayar coba pakai metode bayar kartu bayar seringkali tidak bisa pesan otomatis tunggu bayar beberapa kali jadi minta sama saudara bisa booked aplikasi susah', 'aplikasi kurang nyaman guna pesan masuk tahap bayar tidak bisa batal batal otomatis juka waktu bayar habis soal kadang masuk tahap bayar lihat promo mana bisa guna', 'harga inap hotel pilih lebih mahal waktu coba cek harga pihak hotel langsung jauh lebih murah selisih harga online coba daftar fitur bayar tidak approval alas tidak jelas lanjut kalau memang bukan kriteria ingin jangan muncul icon fitur bayar sangat kecewa', 'sangat kecewa mau pesan tiket tidak bisa bisa stuk lanjut bayar tidak bisa klik ngadu sama sekali tidak guna tidak bisa bantu payah', 'sangat buruk aju fitur bayar belit belit minta data data hasil diinput aju tolak jelas alas tolak jujur applikasi paling ribe

In [23]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, random_state=50)

In [24]:
print(len(train_data))
print(len(test_data))
print(len(train_label))
print(len(test_label))

752
188
752
188


In [25]:
'''
!pip install sklearn-deltatfidf
from sklearn_deltatfidf import DeltaTfidfVectorizer
v = DeltaTfidfVectorizer()
v.fit_transform(data, label)
'''

'\n!pip install sklearn-deltatfidf\nfrom sklearn_deltatfidf import DeltaTfidfVectorizer\nv = DeltaTfidfVectorizer()\nv.fit_transform(data, label)\n'

In [26]:
!pip install sklearn-deltatfidf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


C=0.01

In [27]:

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="linear", C=0.01))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.6861702127659575
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.69      1.00      0.81       129
     Positif       0.00      0.00      0.00        59

    accuracy                           0.69       188
   macro avg       0.34      0.50      0.41       188
weighted avg       0.47      0.69      0.56       188



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


C=0.1

In [28]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="linear", C=0.1))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.8191489361702128
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.79      1.00      0.88       129
     Positif       1.00      0.42      0.60        59

    accuracy                           0.82       188
   macro avg       0.90      0.71      0.74       188
weighted avg       0.86      0.82      0.79       188



C=1

In [29]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="linear", C=1))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.8882978723404256
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.87      0.98      0.92       129
     Positif       0.95      0.68      0.79        59

    accuracy                           0.89       188
   macro avg       0.91      0.83      0.86       188
weighted avg       0.90      0.89      0.88       188



C=10

In [30]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="linear", C=10))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.8563829787234043
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.89      0.90      0.90       129
     Positif       0.78      0.76      0.77        59

    accuracy                           0.86       188
   macro avg       0.83      0.83      0.83       188
weighted avg       0.86      0.86      0.86       188



C=100

In [31]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="rbf", C=100))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.8829787234042553
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.88      0.95      0.92       129
     Positif       0.88      0.73      0.80        59

    accuracy                           0.88       188
   macro avg       0.88      0.84      0.86       188
weighted avg       0.88      0.88      0.88       188



C=1000

In [32]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn_deltatfidf import DeltaTfidfVectorizer

pipe = Pipeline([
      ('vectorizer', DeltaTfidfVectorizer()),
      ("linear svc", SVC(kernel="linear", C=1000))
  ])

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# train and evaluate pipeline SVC in the test dataset
model = pipe
model.fit(train_data, train_label)
y_pred = model.predict(test_data)
print("Accuracy Score: ", accuracy_score(test_label, y_pred))
print("Classification Report:\n ", classification_report(test_label, y_pred))

Accuracy Score:  0.8617021276595744
Classification Report:
                precision    recall  f1-score   support

     Negatif       0.92      0.88      0.90       129
     Positif       0.75      0.83      0.79        59

    accuracy                           0.86       188
   macro avg       0.84      0.85      0.84       188
weighted avg       0.87      0.86      0.86       188



In [33]:
'''
### 5. Remove Stopwords

import string, re, requests, csv
from nltk import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
'''
'''
# CONSTRUCT STOPWORDS
rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword, 
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# CUSTOM STOPWORDS
other = 'admin mimin min minkes kalo nya username'

# gabungkan stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

st_words = set(stopwords_l)
other_stopword = set(other.split())

stop_words = st_words | other_stopword

def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def preprocessing(data):
  data = remove_stopword(data)

  return data

df['Stopword'] = df['KBBI'].apply(lambda x: preprocessing(x))
df

'''

'\n# CONSTRUCT STOPWORDS\nrama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"\nyutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"\nfpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"\nsastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"\naliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"\npebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"\nelang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"\nnltk_stopword = stopwords.words(\'indonesian\')\n